In [ ]:
import os
import pandas as pd
import numpy as np
from data_processor import *
from lda_model import LDATopicModel
from classifier import TopicClassifier
from visualizer import *
from brain_visualizer import *

df_nacc_lda_results =  pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/copathology_final_script/inference_results/260121_nacc_inference.csv')
df_adni_lda_results =  pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/copathology_final_script/inference_results/260121_adni_inference.csv')
df_adni4_lda_results =  pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/copathology_final_script/inference_results/260121_adni4_inference.csv')
df_nacc_resilience = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/nacc/NACC_resilience_inference.csv')
df_adni4_resilience = pd.read_csv('C:/Users/BREIN/Desktop/copathology_visualization_temp/data/adni/ADNI4_resilience_inference.csv')

print(df_nacc_lda_results.shape)
print(df_adni_lda_results.shape)
print(df_adni4_lda_results.shape)

print(df_nacc_resilience.shape)
print(df_adni4_resilience.shape)

df_adni4_resilience = df_adni4_resilience.rename(columns={"FULL_ID": "SUBJ_ID"})
df_nacc_resilience = df_nacc_resilience.rename(columns={"subject_id" : "SUBJ_ID"})

In [ ]:
df_nacc_lda_results = df_nacc_lda_results.merge(
    df_nacc_resilience[["SUBJ_ID", "TN_group"]],
    on="SUBJ_ID",
    how="left"
)
df_adni4_lda_results = df_adni4_lda_results.merge(
    df_adni4_resilience[["SUBJ_ID", "TN_group"]],
    on="SUBJ_ID",
    how="left"
)

In [ ]:
df_nacc_lda_results = df_nacc_lda_results.dropna(subset=['TN_group'])
df_adni4_lda_results = df_adni4_lda_results.dropna(subset=['TN_group'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
prob_cols = df_nacc_lda_results.loc[:,'P(AD)':'P(svPPA)'].columns
group_col = 'TN_group'
# ------------------------------------------------------------
# Compute group-wise mean probabilities
# ------------------------------------------------------------
group_means = (
    df_nacc_lda_results
    .groupby('TN_group')[prob_cols]
    .mean()
)

# ------------------------------------------------------------
# Plot heatmap
# ------------------------------------------------------------
plt.figure(figsize=(8, 5))

sns.heatmap(
    group_means,
    cmap="Reds",
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    cbar_kws={"label": "Mean predicted probability"}
)

plt.xlabel("Predicted pathology")
plt.ylabel("Subgroup")
plt.title("Group-wise Mean Predicted Probability Distribution")

plt.tight_layout()
plt.show()


In [ ]:
# ------------------------------------------------------------
# Convert to long format
# ------------------------------------------------------------
plot_df = df_nacc_lda_results[[group_col] + prob_cols].melt(
    id_vars=group_col,
    value_vars=prob_cols,
    var_name="Pathology",
    value_name="Probability"
)

# ------------------------------------------------------------
# Violin plot
# ------------------------------------------------------------
plt.figure(figsize=(10, 5))

sns.violinplot(
    data=plot_df,
    x="Pathology",
    y="Probability",
    hue=group_col,
    split=True,
    inner="quartile",
    cut=0
)

plt.ylabel("Predicted probability")
plt.title("Distribution of Predicted Probabilities by Subgroup")

plt.legend(title="Subgroup", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()
